<a href="https://colab.research.google.com/github/rsethi21/na_for_systems_bio/blob/main/manually_scale_kds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/rsethi21/na_for_systems_bio.git

Cloning into 'na_for_systems_bio'...
remote: Enumerating objects: 1199, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 1199 (delta 39), reused 38 (delta 20), pack-reused 1112
Receiving objects: 100% (1199/1199), 43.93 MiB | 9.07 MiB/s, done.
Resolving deltas: 100% (683/683), done.
Updating files: 100% (91/91), done.


In [ ]:
!pip3 install -r na_for_systems_bio/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 704.0 kB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, VBox, Button
import argparse
import pdb
import json
import os
import sys
sys.path.append(f"{sys.path[0]}/na_for_systems_bio")

from src.substrate import Substrate
from src.interaction import Interaction
from src.rate import Rate
from src.network import Network
from src.parse import parse_rates, parse_substrates, parse_interactions
from src.score import error

In [ ]:
rates = parse_rates("na_for_systems_bio/input/input_expanded/rates.csv")
substrates = parse_substrates("na_for_systems_bio/input/input_expanded/substrates.csv", rates)
interactions = parse_interactions("na_for_systems_bio/input/input_expanded/interactions.csv", rates, substrates)
network = Network("example", rates, interactions, substrates)

In [ ]:
network.parameters["Km1"].fixed = False

In [ ]:
time = np.array([i for i in range(1001)])
derivatives = network.get_representation_dydt()
for l, t in derivatives.items():
    print(f"{l} = {t}")
print()

d[AKT]/dt = k3[pAKT] - r3[AKT][(PIP3^n2 + Km2^n2)/Km2^n2]
d[pAKT]/dt = r3[AKT][(PIP3^n2 + Km2^n2)/Km2^n2] - k3[pAKT]
d[PTEN]/dt = k4[pPTEN] - r4[PTEN][GSK3B] + s2[LPS][pPTEN]
d[pPTEN]/dt = r4[PTEN][GSK3B] - k4[pPTEN] - s2[LPS][pPTEN]
d[PIP2]/dt = k2[PIP3] - r2[PIP2][PI3K][Km1^n1/(PTEN^n1 + Km1^n1)]
d[PIP3]/dt = r2[PIP2][PI3K][Km1^n1/(PTEN^n1 + Km1^n1)] - k2[PIP3]
d[PI3K]/dt = k1[PI3Ks] - r1[PI3K] + s3[Gio][PI3Ks] - s4[PI3K][LY294-002]
d[PI3Ks]/dt = r1[PI3K] - k1[PI3Ks] - s3[PI3Ks][Gio] + s4[LY294-002][PI3K] + k11[Ca2+]
d[GSK3B]/dt = k5[pGSK3B] - r5[GSK3B][pAKT] + s1[HDACi] - s5[shRNA_GSK3B][GSK3B]
d[pGSK3B]/dt = r5[GSK3B][pAKT] - k5[pGSK3B]
d[TNFa]/dt = k6[PTEN] - r6[TNFa][pAKT]
d[LPS]/dt = 0
d[HDACi]/dt = 0
d[Phagocytosis]/dt = k7[pAKT] - r7[Phagocytosis]
d[LY294-002]/dt = 0
d[ATP]/dt = -r10[ATP]
d[P2Y12act]/dt = k8[P2Y12s][ADP] - r8[P2Y12act] + k10[Ca2+][ADP]
d[P2Y12s]/dt = r8[P2Y12act] - k8[P2Y12s][ADP] - k10[Ca2+][ADP]
d[Gio]/dt = k9[P2Y12act] - r9[Gio][Gio]
d[Ca2+]/dt = -r12[Ca2+]

In [ ]:
with open("na_for_systems_bio/input/input_expanded_2/fitted_params_scaled.json", "r") as fitted_params:
    parameters = json.load(fitted_params)
network.set_parameters(list(parameters.values()), list(parameters.keys()))
network.initial_parameter_values = parameters

In [ ]:
parameters_to_scale = network.find_rate_pairs()
for p_id, p in network.parameters.items():
  if not p.fixed:
    if p_id not in "".join(list(parameters_to_scale.keys())):
      parameters_to_scale[p_id] = p

In [ ]:
sliders = {}
for p, v in parameters_to_scale.items():
  if type(v) == list:
    sliders[p] = FloatSlider(value = 0, min=-10, max=10, step=1, description=f"{p}:")
  else:
    sliders[p] = FloatSlider(value = 0, min=-10, max=10, step=1, description=f"{p}:")

# sliders_visualize = VBox(list(sliders.values()))
# display(sliders_visualize)

In [ ]:
data = json.load(open("/content/na_for_systems_bio/input/input_expanded/fit_data_primary.json"))

In [ ]:
network.reset_stimuli()
for i, stimuli in enumerate(data[1]["stimuli"]):
  network.substrates[stimuli].max_value = data[1]["max_values"][i]
  network.substrates[stimuli].time_ranges = data[1]["time_ranges"][i]

In [ ]:
network.reset_stimuli()
network.substrates["LPS"].max_value = 1.0
network.substrates["LPS"].time_ranges = [[60, 240]]

In [ ]:
data.append({'stimuli': ['LPS'],
  'max_values': [1.0],
  'time_ranges': [[[60, 240]]],
  'substrates': {'pAKT': {'0': 1.0, '30': 1.0, '230': 0.3, '240': 0.3},
   'GSK3B': {'0': 1.0, '30': 1.0, '230': 1.5, '240': 1.5},
   'Phagocytosis': {'0': 1.0, '30': 1.0, '230': 0.35, '240': 0.35},
                 'pPTEN': {'0': 1.0, '30': 1.0, '230':0.1, '240':0.1}}})

In [ ]:
def plot(**updated_parameters):
  time = np.array([i for i in range(1001)])
  updates = {}
  for k, v in updated_parameters.items():
    updates[k] = 10**v
  network.apply_scaling(list(updates.values()), list(updated_parameters.keys()))
  mu, fig = network.graph_distributions(time, 10, substrates_to_plot=["LPS", "HDACi", "pPTEN", "pAKT", "Phagocytosis", "GSK3B"], normalize=True, output_figure=True)
  plt.figure(fig)
  for sub, points in data[-1]["substrates"].items():
    index = list(network.substrates.keys()).index(sub)
    for t, v in data[-1]["substrates"][sub].items():
      plt.plot(int(t), v, color=network.colors[index], marker=".")
  plt.legend(loc="lower right")
  print({p: v.value for p, v in network.parameters.items()})
  network.set_parameters(list(network.initial_parameter_values.values()), list(network.initial_parameter_values.keys()))
  plt.show()

In [ ]:
interact(plot, **sliders)

interactive(children=(FloatSlider(value=0.0, description='k3,r3:', max=10.0, min=-10.0, step=1.0), FloatSlider…

<function __main__.plot(**updated_parameters)>